In [ ]:
pip install kafka-python yfinance pandas

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import yfinance as yf
from datetime import datetime

In [ ]:
producer = KafkaProducer(
    bootstrap_servers=['65.2.198.105:9092'],  # Replace with your Kafka server IP
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

In [ ]:
producer.send('demo_test', value={'rajesh':'meenakshi'})

In [ ]:
ticker_symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']

In [ ]:
def get_multiple_stocks_data(tickers):
    stock_data = yf.download(tickers, period="1d", interval="1m", group_by='ticker')
    latest_data = {}
    
    # Iterate over each ticker to get the most recent record
    for ticker in tickers:
        ticker_data = stock_data[ticker].tail(1).reset_index()
        if not ticker_data.empty:
            # Convert the latest record to a dictionary and add a timestamp
            latest_data[ticker] = ticker_data.to_dict(orient='records')[0]
            latest_data[ticker]['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return latest_data

In [ ]:
while True:
    stock_data = get_multiple_stocks_data(ticker_symbols)

    # Send each stock's data to Kafka
    for ticker, data in stock_data.items():
        producer.send('demo_test', value={ticker: data})
        print(f"Sent data to Kafka for {ticker}: {data}")

    sleep(60) 

In [ ]:
producer.flush() #clear data from kafka server